In [1]:
import pandas as pd
from google.cloud import storage
import mysql.connector
from mysql.connector import Error

In [2]:
BUCKET_NAME = "data_olist_csv"
MYSQL_USR = "root"
MYSQL_PWD = ""
MYSQL_HOST = "34.135.162.156"
MYSQL_DB_NAME = "Olist"

In [3]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    """Lists all the blobs in the bucket that begin with the prefix.

    This can be used to list all blobs in a "folder", e.g. "public/".

    The delimiter argument can be used to restrict the results to only the
    "files" in the given "folder". Without the delimiter, the entire tree under
    the prefix is returned. For example, given these blobs:

        a/1.txt
        a/b/2.txt

    If you specify prefix ='a/', without a delimiter, you'll get back:

        a/1.txt
        a/b/2.txt

    However, if you specify prefix='a/' and delimiter='/', you'll get back
    only the file directly under 'a/':

        a/1.txt

    As part of the response, you'll also get back a blobs.prefixes entity
    that lists the "subfolders" under `a/`:

        a/b/
    """

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)
    
    return [blob.name.replace("deltas_por_cargar/","") for blob in blobs][1:]


In [9]:
list_blobs = list_blobs_with_prefix(BUCKET_NAME,prefix="deltas_por_cargar/")
list_blobs

['item_delta.csv', 'order_delta.csv', 'payment_delta.csv', 'review_delta.csv']

In [5]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name, password=user_password, user=user_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [6]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name, user=user_name, passwd=user_password, database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [7]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [8]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [13]:
def prepare_to_sql(df):
    df = list(df.itertuples(index=False, name=None))
    df_string = ",".join(["(" + ",".join([str(i) for i in df_d]) + ")" for df_d in df])
    return df_string

In [10]:
item_delta = pd.read_csv("gs://data_olist_csv/deltas_por_cargar/item_delta.csv")
order_delta = pd.read_csv("gs://data_olist_csv/deltas_por_cargar/order_delta.csv")
payment_delta = pd.read_csv("gs://data_olist_csv/deltas_por_cargar/payment_delta.csv")
review_delta = pd.read_csv("gs://data_olist_csv/deltas_por_cargar/review_delta.csv")

In [11]:
item_delta.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,9dcc4e7d4978e71c0c8c8562a72935fda,1,47125061c86148a6f08a9e8708cef1ff,c6bda72e4dbf5c5866b13cb1810c6d03,2018-09-01 23:25:09,295.7,13.09
1,9dcfd5e81815f361cac59d41e752483ea,1,af2d506fb01b724e03591349ad36b67e,4b9750c8ad28220fe6702d4ecb7c898f,2018-08-26 23:04:22,48.9,23.49
2,9dd9eb1d7c035198b06f1775e0711a59a,1,3af6d5f9fdb78f106c003ce49d7f0186,42b729f859728f5079499127a9c2ef37,2018-08-10 16:50:17,32.9,7.49
3,9ddf2a1cb3163c05aa196a8e43336bfda,1,eb2e510c7e0f62651d0156e769928970,1d4587203296c8f4ad134dc286fa6db0,2018-08-12 08:45:16,27.9,18.30
4,9de18154654e59d76a22157b8e439adda,1,5d91ebba17efcefccb56680d45aae16b,7df1fc11cc5cbc518b39faa60cf5d67c,2018-08-31 12:24:23,314.9,29.15


In [16]:
"hola " + "mundo"

'hola mundo'

In [14]:
prepare_to_sql(item_delta)

'(9dcc4e7d4978e71c0c8c8562a72935fda,1,47125061c86148a6f08a9e8708cef1ff,c6bda72e4dbf5c5866b13cb1810c6d03,2018-09-01 23:25:09,295.7,13.09),(9dcfd5e81815f361cac59d41e752483ea,1,af2d506fb01b724e03591349ad36b67e,4b9750c8ad28220fe6702d4ecb7c898f,2018-08-26 23:04:22,48.9,23.49),(9dd9eb1d7c035198b06f1775e0711a59a,1,3af6d5f9fdb78f106c003ce49d7f0186,42b729f859728f5079499127a9c2ef37,2018-08-10 16:50:17,32.9,7.49),(9ddf2a1cb3163c05aa196a8e43336bfda,1,eb2e510c7e0f62651d0156e769928970,1d4587203296c8f4ad134dc286fa6db0,2018-08-12 08:45:16,27.9,18.3),(9de18154654e59d76a22157b8e439adda,1,5d91ebba17efcefccb56680d45aae16b,7df1fc11cc5cbc518b39faa60cf5d67c,2018-08-31 12:24:23,314.9,29.15),(9de375e01aa3f39d32001e75abd82b99a,1,45746155b8481c23a21eb4645c306438,0a85ebe4e328db81ac9109781205e2f7,2018-08-08 22:10:38,59.0,8.57),(9de8f4ac771189b55e817ebd04cf21a7a,1,9fe172fa8e662ca8572cf12abf8bce23,cab85505710c7cb9b720bceb52b01cee,2018-09-03 02:44:20,49.9,15.45),(9dfd8df84d4f07cc8da0f2fa0fe96075a,1,ba74c6b75d2ad75031

In [ ]:
conf_mysql = {
    "host_name": MYSQL_HOST,
    "user_name": MYSQL_USR,
    "user_password": MYSQL_PWD,
    "db_name": MYSQL_DB_NAME
}

In [ ]:
db_connection = create_db_connection(
    host_name=MYSQL_HOST,
    user_name=MYSQL_USR,
    user_password=MYSQL_PWD,
    db_name=MYSQL_DB_NAME
)

In [ ]:
item_delta = list(item_delta.itertuples(index=False, name=None))
item_delta

In [ ]:
item_delta_string = ",".join(["(" + ",".join([str(i) for i in itm_d]) + ")" for itm_d in item_delta])
item_delta_string

In [22]:
def move_blob(bucket_name, blob_name, destination_bucket_name, destination_blob_name):
    """Moves a blob from one bucket to another with a new name."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The ID of your GCS object
    # blob_name = "your-object-name"
    # The ID of the bucket to move the object to
    # destination_bucket_name = "destination-bucket-name"
    # The ID of your new GCS object (optional)
    # destination_blob_name = "destination-object-name"

    storage_client = storage.Client()

    source_bucket = storage_client.bucket(bucket_name)
    source_blob = source_bucket.blob(blob_name)
    destination_bucket = storage_client.bucket(destination_bucket_name)

    blob_copy = source_bucket.copy_blob(
        source_blob, destination_bucket, destination_blob_name
    )
    source_bucket.delete_blob(blob_name)

    print(
        "Blob {} in bucket {} moved to blob {} in bucket {}.".format(
            source_blob.name,
            source_bucket.name,
            blob_copy.name,
            destination_bucket.name,
        )
    )


In [13]:
item_string = prepare_to_sql(item_delta)
order_string = prepare_to_sql(order_delta)
payment_string = prepare_to_sql(payment_delta)
review_string = prepare_to_sql(review_delta)

read_query(db_connection,"DESC payment")

read_query(db_connection,"DESC review")

read_query(db_connection,"DESC `order`")

read_query(db_connection,"DESC item")

In [18]:
insert_item = "INSERT INTO item (order_id, order_item_id,\
                product_id, seller_id, shipping_limit_date,\
                price, freight_value) VALUES"+item_string+";"

In [19]:
insert_payment = "INSERT INTO payment (order_id, payment_sequential,\
                payment_type_id, payment_installments, payment_value)\
                VALUES"+payment_string+";"

In [20]:
insert_review = "INSERT INTO review (review_id, order_id,review_score,\
                review_comment_title,review_comment_message,\
                review_creation_date,review_answer_timestamp)\
                VALUES"+review_string+";"

In [21]:
insert_order = "INSERT INTO `order` (order_id,customer_id,order_status_id,\
                order_purchase_timestamp,order_approved_at,\
                order_delivered_carrier_date,order_delivered_customer_date,\
                order_estimated_delivery_date) VALUES"+order_string+";"

In [ ]:
execute_query(db_connection, insert_item)
execute_query(db_connection, insert_order)
execute_query(db_connection, insert_payment)
execute_query(db_connection, insert_review)

In [ ]:
for blob in list_blobs:
    move_blob(BUCKET_NAME,f"/deltas_por_cargar/{blob}",BUCKET_NAME,f"/deltas_cargados/{blob})